<a href="https://colab.research.google.com/github/ZeBread/tester/blob/main/trail_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#PACKAGES
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
#DRIVE LINKING
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#DATA DIRECTORY LINKING
DATADIR = "/content/drive/MyDrive/train_200px"
CATEGORIES = ["FH","MC","MH","NL","OH","VC"]

CREATING DATASET

In [ ]:
batch_size = 16
img_height = 200
img_width = 200

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATADIR,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 592 files belonging to 6 classes.
Using 533 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATADIR,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 592 files belonging to 6 classes.
Using 59 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['FH_200px', 'MC_200px', 'MH_200px', 'NL_200px', 'OH_200px', 'VC_200px']


VISUALISE DATA

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(16, 200, 200, 3)
(16,)


Configure the dataset for performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(50).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Standardize the data

CREATE MODEL

In [ ]:
num_classes = 6

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_8 (Rescaling)      (None, 200, 200, 3)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 200, 200, 16)      448       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 100, 100, 32)      4640      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 25, 25, 64)       

In [ ]:
epochs = 100
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs = epochs
)

Epoch 1/100
34/34 [==============================] - 2s 21ms/step - loss: 12.4598 - accuracy: 0.1594 - val_loss: 13.3862 - val_accuracy: 0.1695
Epoch 2/100
34/34 [==============================] - 0s 13ms/step - loss: 13.6422 - accuracy: 0.1536 - val_loss: 13.3862 - val_accuracy: 0.1695
Epoch 3/100
34/34 [==============================] - 0s 13ms/step - loss: 13.3090 - accuracy: 0.1743 - val_loss: 13.3862 - val_accuracy: 0.1695
Epoch 4/100
34/34 [==============================] - 0s 13ms/step - loss: 13.3567 - accuracy: 0.1713 - val_loss: 13.3862 - val_accuracy: 0.1695
Epoch 5/100
34/34 [==============================] - 0s 13ms/step - loss: 13.7262 - accuracy: 0.1484 - val_loss: 13.3862 - val_accuracy: 0.1695
Epoch 6/100
34/34 [==============================] - 0s 13ms/step - loss: 13.3099 - accuracy: 0.1742 - val_loss: 13.3862 - val_accuracy: 0.1695
Epoch 7/100
34/34 [==============================] - 0s 13ms/step - loss: 13.5147 - accuracy: 0.1615 - val_loss: 13.3862 - val_accuracy:

In [143]:
keras_file = "model.h5"
keras.models.save_model(model, keras_file)

In [146]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpo4wygceg/assets


INFO:tensorflow:Assets written to: /tmp/tmpo4wygceg/assets


In [147]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
